In [ ]:
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [ ]:
FAILURE = -1
SUCCESS = 0
GPIO_OUT = 0
tempPin = 1
microsec = 1000000

from enum import Enum
class GPIO(Enum):
    LOW = 0
    HIGH = 1

In [ ]:

#creat Pmod_IO object to be used to read values. Wired to + of buzzer sensor to PMODB
#Pins 0,1 and 4,5 are connected to pins with pull-down resistors
#read_pin = Pmod_IO(base.PMODB,tempPin,'in')
#write_pin = Pmod_IO(base.PMODB,tempPin,'out')

In [ ]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"
#include "stdint.h"
#define MAXTIMINGS  85
#define DHTPIN      1
#define HIGH 1
#define LOW 0
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}


int dht11_dat[5] = { 0, 0, 0, 0, 0 };
void read_dht11_dat()
{
    uint8_t laststate  = HIGH;
    uint8_t counter    = 0;
    uint8_t j          = 0, i;
    float f; /* fahrenheit */
    dht11_dat[0] = dht11_dat[1] = dht11_dat[2] = dht11_dat[3] = dht11_dat[4] = 0;
    /* pull pin down for 18 milliseconds */
    write_gpio(DHTPIN,0);
    sleep(0.018);
    write_gpio(DHTPIN,1);
    usleep(40);
    /* then pull it up for 40 microseconds */
    write_gpio(DHTPIN,1);
    usleep(40);
    /* prepare to read the pin */
    read_gpio(DHTPIN);
    /* detect change and read data */
    for ( i = 0; i < MAXTIMINGS; i++ )
    {
        pyprintf( "Data not good, skip\n" );
        counter = 0;
        while ( read_gpio(DHTPIN) == laststate )
        {
            counter++;
            usleep( 1 );
            if ( counter == 255 )
            {
                break;
            }
        }
        laststate = read_gpio( DHTPIN );
        if ( counter == 255 )
            break;
        /* ignore first 3 transitions */
        if ((i >= 4) && (i % 2 == 0))
        {
            /* shove each bit into the storage bytes */
            dht11_dat[j / 8] <<= 1;
            if (counter > 16)
                dht11_dat[j / 8] |= 1;
            j++;
        }
    }
    /*
     * check we read 40 bits (8bit x 5 ) + verify checksum in the last byte
     * print it out if data is good
    */
    if ( (j >= 40) &&
         (dht11_dat[4] == ( (dht11_dat[0] + dht11_dat[1] + dht11_dat[2] + dht11_dat[3]) & 0xFF) ) )
    {
        f = dht11_dat[2] * 9. / 5. + 32;
        pyprintf( "Humidity = %d.%d %% Temperature = %d.%d *C (%.1f *F)\n",
            dht11_dat[0], dht11_dat[1], dht11_dat[2], dht11_dat[3], f );
    }
    else  
    {
        pyprintf( "Data not good, skip\n" );
    }
}

void run()
{
   
    {
        read_dht11_dat();
        sleep(1); 
    }
}

In [ ]:
while True:
    read_dht11_dat()
    time.sleep(1);

In [ ]:
"""
read temperature and humidity from KY-015 datasheet sample
"""

def bin2dec(string_num):
    return str(int(string_num, 2))

def read_data():
    data = 0
    for i in range(0,8):
        if (read_gpio(tempPin) == GPIO.LOW):
            while (read_gpio(tempPin) == GPIO.LOW): # wait for 50us
                pass
            time.sleep(30/microsec) # determine the duration of the high level to determine the data is '0 'or '1'
            if (read_gpio(tempPin) == GPIO.HIGH):
                data |= (1 << (7-i)) # high front and low in the post
            while (read_gpio(tempPin) == GPIO.HIGH): # data '1 ', wait for the next one receiver
                pass
    return data

while True:
    write_gpio(tempPin,0) # bus down, send start signal
    time.sleep(30/microsec); # delay greater than 18ms, so DHT11 start signal can be detected

    write_gpio(tempPin,1)
    time.sleep(40/microsec); # Wait for DHT11 response

    while (read_gpio(tempPin) == GPIO.HIGH):
        pass
    time.sleep(80/microsec) # DHT11 response, pulled the bus 80us
    if (read_gpio(tempPin) == GPIO.LOW): # copied from C version (???)
        pass
    time.sleep(80/microsec) # DHT11 80us after the bus pulled to start sending data

    dat = [0.0,0.0,0.0,0.0]
    for i in range(0,4): # receive temperature and humidity data, the parity bit is not considered
        dat[i] = read_data()

    write_gpio(tempPin,1) # send data once after releasing the bus, wait for the host to open the next Start signal
    
    # print results
    print('Current humidity: %3d.%d' % (dat[0], dat[1]))
    print('Current temperature: %3d.%dC' % (dat[2], dat[3]))
    time.sleep(1)

In [ ]:

#circuitbase.com

def bin2dec(string_num):
    return str(int(string_num, 2))
data = []
write_gpio(tempPin,1)
time.sleep(0.025)
write_gpio(tempPin,0)
time.sleep(0.02)
read_gpio(tempPin)
for i in range(0,500):
    data.append(read_gpio(tempPin))
bit_count = 0
tmp = 0
count = 0
HumidityBit = ""
TemperatureBit = ""
crc = ""
try:
	while data[count] == 1:
		tmp = 1
		count = count + 1
	for i in range(0, 32):
		bit_count = 0
		while data[count] == 0:
			tmp = 1
			count = count + 1
		while data[count] == 1:
			bit_count = bit_count + 1
			count = count + 1
		if bit_count > 3:
			if i>=0 and i<8:
				HumidityBit = HumidityBit + "1"
			if i>=16 and i<24:
				TemperatureBit = TemperatureBit + "1"
		else:
			if i>=0 and i<8:
				HumidityBit = HumidityBit + "0"
			if i>=16 and i<24:
				TemperatureBit = TemperatureBit + "0"
except:
	print ("ERR_RANGE")
try:
	for i in range(0, 8):
		bit_count = 0
		while data[count] == 0:
			tmp = 1
			count = count + 1
		while data[count] == 1:
			bit_count = bit_count + 1
			count = count + 1
		if bit_count > 3:
			crc = crc + "1"
		else:
			crc = crc + "0"
except:
	print("ERR_RANGE")
Humidity = bin2dec(HumidityBit)
Temperature = bin2dec(TemperatureBit)
if int(Humidity) + int(Temperature) - int(bin2dec(crc)) == 0:
	print("Humidity: {} %".format(Humidity))
	print("Temperature: {} C".format(Temperature))
else:
	print("ERR_CRC")

In [ ]:
#from circuit basics 2nd example
dht11_dat = [int(0), int(0), int(0), int(0), int(0),int(0),int(0),int(0),int(0)]
MAXTIMINGS = 85
DHTPIN = 2

def read_dht11_dat():
    laststate = 1
    counter = 0
    j = int(0)
    i = int(0)
    f = float()
    
    dht11_dat[0] = dht11_dat[1] = dht11_dat[2] = dht11_dat[3] = dht11_dat[4] = int(0)
    write_gpio(DHTPIN,0)
    time.sleep(18/1000)
    write_gpio(DHTPIN,1)
    time.sleep(40/1000000)
    
    for i in range(0,MAXTIMINGS):
        counter = 0
        while read_gpio(DHTPIN) == laststate:
            counter +=1
            time.sleep(1/1000000)
            if counter == 255:
                break
        laststate = read_gpio(DHTPIN)
        if counter == 255:
            break
        if (i >= 4) and (i % 2 == 0):
            dht11_dat[int(j / 8)] <<= 1
            if counter > 16:
                dht11_dat[int(j / 8)] |= 1
            j +=1
    if (j >= 40) and (dht11_dat[4] == ((dht11_dat[0] + dht11_dat[1] + dht11_dat[2] + dht11_dat[3]) & 0xFF)):
        f = dht11_dat[2] * 9. / 5. + 32
        print("Humidity: {}.{}".format(dht11_dat[0], dht11_dat[1]))
        print("Temp: {} C".format(dht11_dat[2])); #Uncomment for Celsius
        print("Temp: {} F".format(f)) #Comment out for Celsius
    else:
        f = dht11_dat[2] * 9. / 5. + 32
        print("Fault Humidity: {}.{}".format(dht11_dat[0], dht11_dat[1]))
        print("Falut Temp: {} C".format(dht11_dat[2])); #Uncomment for Celsius
        print("Fault Temp: {} F".format(f)) #Comment out for Celsius
                
            
    
    


In [ ]:
while True:
    read_dht11_dat()
    time.sleep(1); 


In [ ]:
"""
from enum import Enum
class GPIO(Enum):
    LOW = 0
    HIGH = 1
"""
read temperature and humidity from KY-015
"""

def bin2dec(string_num):
    return str(int(string_num, 2))

def read_data():
    data = 0
    for i in range(8):
        if (read_pin.read() == GPIO.LOW):
            while (read_pin.read() == GPIO.LOW): # wait for 50us
                pass
            time.sleep(30 * 1e-6) # determine the duration of the high level to determine the data is '0 'or '1'
            if (read_pin.read() == GPIO.HIGH):
                data |= (1 << (7-i)) # high front and low in the post
            while (read_pin.read() == GPIO.HIGH): # data '1 ', wait for the next one receiver
                pass
    return data

while True:
    write_pin.write(0) # bus down, send start signal
    time.sleep(30 * 1e-6); # delay greater than 18ms, so DHT11 start signal can be detected

    write_pin.write(1)
    time.sleep(40 * 1e-6); # Wait for DHT11 response

    while (read_pin.read() == GPIO.HIGH):
        pass
    time.sleep(80 * 1e-6) # DHT11 response, pulled the bus 80us
    if (read_pin.read() == GPIO.LOW): # copied from C version (???)
        pass
    time.sleep(80 * 1e-6) # DHT11 80us after the bus pulled to start sending data

    dat = [0,0,0,0]
    for i in range(0,4): # receive temperature and humidity data, the parity bit is not considered
        dat[i] = read_data()

    write_pin.write(1) # send data once after releasing the bus, wait for the host to open the next Start signal
    
    # print results
    print('Current humidity: %3d.%d' % (dat[0], dat[1]))
    print('Current temperature: %3d.%dC' % (dat[2], dat[3]))
    time.sleep(1)